In [140]:
import numpy as np
import pandas as pd

raw = pd.read_csv('./nyse/prices-split-adjusted.csv')
s_raw = pd.read_csv('./nyse/Fundamentals with Ratios FINAL/fundementals_with_ratios_testing_set.csv')
till_year = "2014-09-01"
arr_size = 1172
#pre-defined number of price data found from the data. Used to make sure there are equal number of prices for each stock to calculate covariance. We will discard the rest.

In [60]:
raw.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [61]:
s_raw.head()

,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,...,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding
0,AAL,12/31/2014,4668000000,-160000000,-150000000,143,-5311000000,15135000000,60,1768000000,...,4.322500e+10,1.175000e+10,13404000000,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.000000e+00,4.02,7.169154e+08
1,AAPL,9/27/2014,48649000000,-6452000000,980000000,35,-9571000000,0,40,13844000000,...,2.320000e+11,6.853100e+10,63448000000,1.120000e+11,1.200000e+11,2.320000e+11,1.830000e+11,0.000000e+00,6.49,6.087827e+09
2,ABBV,12/31/2014,6954000000,-172000000,-651000000,102,-612000000,4194000000,74,8348000000,...,2.751300e+10,1.608100e+10,11393000000,1.742000e+09,2.577100e+10,2.751300e+10,1.996000e+10,-9.720000e+08,1.11,1.598198e+09
3,ABC,9/30/2014,17250160000,-938286000,-28594000,14,-264457000,2749185000,10,1808513000,...,2.153218e+10,1.680020e+10,17250160000,1.956899e+09,1.957528e+10,2.153218e+10,1.200000e+11,-2.313380e+09,1.22,2.247787e+08
4,ABT,12/31/2014,5350000000,-195000000,69000000,11,-1077000000,0,43,4063000000,...,4.120700e+10,1.355600e+10,10467000000,2.152600e+10,1.968100e+10,4.120700e+10,2.024700e+10,-8.678000e+09,1.50,1.522667e+09


In [145]:
stock_symbols = s_raw['Ticker Symbol'].unique()
stock_symbols.shape

returns = pd.DataFrame()

In [146]:
def calculate_stats():
    for s in stock_symbols:
        c = raw.loc[raw['symbol']==s] #filter by company symbol
        c = c.loc[c['date']<till_year] #only take entries before till_year
        if not c.empty:
            prices = c['close'].values #store close values as np array
            exp_ret = calculate_returns(prices)
            print(exp_ret)
            if exp_ret.size >= arr_size: 
                returns[s] = exp_ret[:arr_size]
            #pd.concat([returns,prices], ignore_index=True, axis=1)
            print(s, exp_ret.size)
            
            #break

In [147]:
def calculate_returns(prices):
    prices_shifted = np.roll(prices, -1) #shift values left by 1
    e = ((prices_shifted/prices)-1)*100 #calculate daily percent return 
    e = e[:-1] #remove last element to counteract shift
    return e
    

In [148]:
calculate_stats()

[11.32075472 -4.14312618  2.94695481 ... -0.25581479  0.46164655
 -0.7403549 ]
AAL 1172
[ 0.17288678 -1.59063257 -0.18485804 ...  1.22905938  0.11750025
  0.24449878]
AAPL 1172
[-0.8257318  -1.26328732  0.20354755 -2.17643651  0.56363989  0.29497933
 -0.44118235  0.70901629  1.49603403  2.71677183  2.47607478  2.47117531
 -2.41158092  3.78912981 -0.89947092  0.37372932 -1.88829531  1.13852538
 -1.01849635 -0.64988086  1.33551653  0.26895912 -0.21458692 -0.16129569
 -1.93861341 -0.46677103 -1.10345379 -1.19944219 -0.42348393  3.685852
  2.7618321   1.62319576  1.0997696   0.44029525 -0.82516763 -2.83411344
 -0.74926146 -0.97061202  0.51728288  2.41062581  1.13726789 -1.90900618
  0.61318052 -0.6624271  -0.40011201  0.8837654  -1.32731624  0.
  1.04924138  2.50265968 -0.88311688  0.52411164  1.82481747 -1.4592908
  1.2470745   0.89812418  2.2634766  -0.14920916  1.56910827  0.83374205
 -0.99708174 -0.3438934   2.53881686 -0.69710821  1.11352938  1.86736653
  2.51468866 -0.96285651 -0.254

[ 1.65669815  0.52059756 -0.27020716 ...  0.20284483 -0.02024696
 -0.18222414]
AMT 1172
[ 0.58999853 -1.81156876 -1.70132325 ...  0.39493492 -0.92080076
 -0.28821247]
AMZN 1172
[ 1.55682408  3.83239653  1.77165354 ... -0.20344183 -0.20385656
  0.74280783]
AN 1172
[ 1.18069327  1.56833453 -1.51654499 ... -0.01969867 -0.15758495
  0.45378217]
APA 1172
[ 2.81124507  0.87139576 -0.16383676 ... -1.38331366  0.52488869
  1.44940581]
APC 1172
[-0.82972583 -0.82454711 -0.57464485 ... -0.29985457  0.25564362
 -0.09749737]
APD 1172
[-2.08968006  0.3779502  -0.66446066 ... -0.26016285 -0.11593372
 -0.36753748]
APH 1172
[-3.12312068  5.20768334 -2.12138317 ...  0.72333277 -0.83783293
  0.24141202]
ARNC 1172
[ 0.17699115 -0.53003534 -2.39786856 ...  0.78227292  0.68994822
  0.81370878]
ATVI 1172
[ 1.01544812  0.52447992  0.21738696 ...  1.51118272 -0.35728463
  0.81673305]
AWK 1172
[0.83264782 1.43131289 3.12076543 ... 0.61869263 0.09708981 0.12932105]
AYI 1172
[-0.83527938 -0.93803965  1.32697625 

[-0.09596017  0.06002521 -0.88792661 ... -0.17775436 -0.17807089
 -0.28882433]
DNB 1172
[-0.47214116  1.18595345  2.20346929 ... -0.69569455 -0.45197966
 -0.2610613 ]
DOV 1172
[-1.16838488  2.22531293 -1.08843537 ...  1.54401936  1.08379489
  0.68810367]
DPS 1172
[-0.42857714 -1.31993982  0.93050308 ... -0.33606385 -0.63224235
  0.36054717]
DRI 1172
[-1.64997229  0.35950308  0.47761243 ...  0.84347345  0.52104484
  0.92756789]
DUK 1172
[-0.85113154 -0.58912473  0.86352691 ...  0.1880962  -0.20115596
  0.34937115]
DVA 1172
[ 0.10448217 -0.30007044 -0.58884718 ... -0.37553515  0.47118741
  1.05855692]
DVN 1172
[ 2.80991736 -2.57234727 -0.77007151 ... -0.90521829 -0.32241536
  1.99461466]
EA 1172
[-1.0460251  -0.63424947 -1.14894468 ... -0.58666848 -0.87624827
  0.12628361]
EBAY 1172
[-1.52485967 -0.29174371  1.08035111 ... -0.68773481  0.2717365
  0.37590786]
ECL 1172
[-1.30013219  1.07166553 -0.35343274 ...  0.79379081  1.0325534
  0.27715053]
ED 1172
[-0.0317763   1.08073748  0.4088081

[-2.70470545  0.45696497 -1.00454515 ... -0.16147021  0.
  0.4285929 ]
FFIV 1172
[ 4.28211587 -0.56360306 -0.56680969 ... -0.55846249 -0.47385223
  0.07053606]
FIS 1172
[ 0.42484119 -0.42304392 -0.46530245 ... -0.41900061  0.28050492
  0.18648485]
FISV 1172
[ 2.77777778  3.83609416 -0.75566751 ...  1.75185302 -0.62122471
  0.21432219]
FL 1172
[ 0.24510417  0.24449266  0.12195427 ... -0.61783756 -0.35523684
  0.38621806]
FLIR 1172
[-0.06519887  2.60983036  3.79398262 ... -0.45104273 -0.74626598
 -0.79216431]
FLR 1172
[ 1.11328937  2.76276379  2.74802494 ... -0.49835672  0.15815869
 -0.13159495]
FLS 1172
[-0.33438754  0.97121489 -1.24169115 ...  0.48929509  0.63907181
  0.        ]
FMC 1172
[ 1.43214742  1.9068487   0.32851877 ... -1.43621099 -1.21428286
  0.76644683]
FSLR 1172
[-0.6281407  -1.39064475 -0.51282051 ...  0.74738416  0.14836795
  0.59259259]
FTR 1172
[ 0.15898395 -0.0865873   0.28885616 ... -0.3295852  -0.48390919
 -0.11346057]
GD 1172
[-0.09238106  3.46740654  0.24575738 .

[-0.10490689 -0.18377527  0.55234087 ... -0.02277044 -0.39864008
  0.33161922]
GPC 1172
[-0.20884185 -0.36149543 -0.63012795 ... -0.193182   -0.1935573
  0.7341723 ]
GPN 1172
[ 1.21713723  1.20250114 -3.6597004  ... -0.19451048  0.32482027
 -0.38851714]
GPS 1172
[ 0.2485954  -0.74394921  0.46845097 ... -4.99477732 -0.86096538
  0.38803216]
GRMN 1172
[ 6.49087221  0.50793651 -1.38976627 ...  1.51214093  1.01920815
  0.77609236]
GT 1172
[ 0.04113636  0.38034848  0.94213823 ... -1.12142493 -0.1788626
  0.26062836]
GWW 1172
[ 1.28        2.36967457  0.24690739 ... -1.90516407 -1.6063171
  0.34135353]
HAL 1172
[-1.10633371 -0.09321939 -1.43079618 ...  0.05743442 -0.26788941
  1.03607254]
HAS 1172
[ 1.11821409  0.91627488  1.18972758 -1.91831374  4.25866864  0.36308322
  0.09045222  1.95782223  3.36781113  0.54300654  3.8658357   1.12205799
  2.84167516 -1.15789211  0.45260648  0.02650941 -0.95389767 -1.92616635
  3.84615385 -1.10322564  1.51394149 -1.59602298 -0.2658947  -0.87976011
 -0.295

...

[ 1.6273049   1.53089227  1.12651125 ... -0.55619171  0.52640236
  2.22549497]
MUR 1172
[ 2.59492212  0.94515305 -1.34511931 ... -0.67227449  0.
  1.72024963]
NBL 1172
[-0.97652582  0.98615589 -0.2816939  ...  0.17576303  0.66054289
  0.94329746]
NEE 1172
[ 0.26831991  2.12021193 -0.64503123 ...  0.0754717   1.28205505
  0.85628813]
NEM 1172
[-3.68361637  3.51387685 -1.72542764 ... -0.97213488  0.10744154
  0.51134486]
NFLX 1172
[ 1.46234781  0.96741753 -1.48611266 ... -0.20352781 -0.86109447
  2.44571429]
NFX 1172
[-0.48402711  1.134893    1.15420965 -1.14103966 -0.54504968 -0.3868472
  0.84142395 -0.06418806  0.57803791 -0.70242656  0.57878135  1.1189194
 -0.41099906  0.63492381  1.51419238 -0.12430392  1.24456133 -0.09220344
  0.09228853 -0.09220344  0.09228853 -1.75169632  1.06350332 -0.03095636
  1.26934988  0.30572303 -0.42670831  1.80593828  1.3830518  -3.46975672
 -0.46082029  0.89505241 -1.34597435 -1.17829147  1.22371192  0.52697461
  1.66512797  2.42644518 -0.23689962  1.068

[-0.35335438 -0.78521277 -0.58718155 ...  0.55624249 -0.12445997
 -0.29077679]
OMC 1172
[ 0.15584675 -0.44087654 -0.3125788  ...  0.23234865  0.74694396
 -0.30679469]
ORLY 1172
[-0.72201801  1.19999988 -0.74261222 ...  0.22297235  0.60940315
 -0.26920583]
OXY 1172
[-0.06489617  0.81168834  1.57810633 ... -0.04764412 -0.42897999
 -0.31114649]
PAYX 1172
[-3.92670157  0.99909173  0.13489209 ... -0.03703704  0.07410152
  0.18511292]
PBI 1172
[ 1.80278334  1.31472498  1.32415261 ... -0.0956512   0.03191479
  0.19142128]
PCAR 1172
[-1.12359551  1.15908636 -1.70747705 ...  0.85883946  0.7205284
  0.75872316]
PCG 1172
[ 1.16091754 -3.30155194 -1.3465387  ... -1.31113405 -0.79792974
 -0.51170153]
PCLN 1172
[-0.34710171 -0.06965517  2.16102467 ... -0.79070668  0.4234072
 -0.12400794]
PDCO 1172
[-0.08921498 -0.53571432 -1.34649021 ...  1.40379293  0.48859935
  0.99945705]
PEG 1172
[ 1.20835724 -1.00032107 -0.63559321 ... -0.39956264  0.37948497
 -0.09721754]
PEP 1172
[-1.42630745 -0.32154341 -0.3

[ 4.15175853  1.39747073 -0.79077726 ... -0.27258209 -0.47210462
 -0.49931593]
SNI 1172
[-2.01503158  1.41190599 -0.42372577 ...  0.66605648  0.91261232
  0.38435903]
SO 1172
[ 0.0892532  -0.89179254  2.21956209 ...  0.36113697  0.34744635
  0.32149622]
SPGI 1172
[0.08163265 0.24469821 1.30187144 ... 3.88788427 1.47954743 0.17152659]
SPLS 1172
[-0.8883285  -0.07316079  0.16474464 ... -0.2267739   0.0084199
  0.04208333]
SRCL 1172
[-2.12880672 -1.37086458 -0.2223851  ...  1.43456718  0.90436308
  1.03928683]
SRE 1172
[-1.95870312 -0.70194924 -1.90319749 ...  0.85723706  1.48741419
  0.78918022]
STX 1172
[ 1.55195682 -0.93023256 -1.14017438 ...  0.29250457  1.23951875
  2.0165628 ]
SWKS 1172
[ 0.93942101  1.53845578  3.34829973 ... -0.6276355   0.41296732
  0.77416235]
SYK 1172
[-0.5434837   0.3278798  -0.21787036 ... -0.45230674 -0.20652211
  0.49669288]
SYMC 1172
[-0.42704626  0.28591494  0.10691732 ... -0.02648133  0.0794571
  0.10585075]
SYY 1172
[-0.48984955 -2.91842465 -1.1227888  

[-5.66123188e-01  4.78248691e-01  2.26654584e+00 -8.86520410e-01
 -8.72090768e-01 -1.89487923e+00  1.12669114e+00  3.86543884e-01
  0.00000000e+00 -4.07703266e-01  2.72911070e-01 -9.29916081e-01
  1.46520147e+00  1.17328520e+00 -4.46030330e-02  2.54350513e+00
  1.17493693e+00 -1.29031828e+00 -8.27891031e-01  1.66959359e+00
  4.32152127e-02 -1.42548599e+00  2.62927482e+00  3.41588386e-01
 -3.19153191e-01  3.62864477e-01 -1.68013611e+00  9.30131949e-01
  5.14363480e-01 -7.14286323e+00  1.99770154e+00 -2.02610995e+00
 -1.74632813e+00  4.20956521e-01  2.11923386e+00 -4.78903101e-01
  2.06235110e-01  3.52160988e+00  5.96423680e-01  1.75665345e-01
 -2.41124512e-01  4.76818742e+00  1.99245176e+00  4.03043175e+00
  8.69735126e-01  7.25069581e-01 -4.86381304e-01  2.42423646e+00
  3.81754159e-01 -3.61281612e-01  5.72480894e-02 -2.47951554e-01
  1.79732891e+00 -2.79865128e+00  1.44927536e+00 -4.95234286e-01
  3.82842635e-01 -4.57662099e-01  6.13024892e-01 -1.56130807e+00
 -2.30174657e+00 -2.77172

[-1.30705409  1.01874486 -0.20169826 ...  1.22532562  1.18806991
  0.42091937]
WEC 1172
[-0.82584919  1.1585807  -0.42949533 ...  3.40347345 -1.53266336
 -0.12758357]
WFM 1172
[-0.20744113 -0.88041209  2.14656433 ...  0.55383201  0.
  0.33441021]
WHR 1172
[-0.43911593 -0.02939724  0.2353     ...  0.83744685 -0.29812394
  0.32037804]
WM 1172
[ 1.24945691  3.24496082 -0.70826622 ...  0.47586506 -0.59201456
  1.14003573]
WMB 1172
[-0.99576065 -0.22350345  0.05599776 ...  0.43697168  0.06592485
 -0.52701184]
WMT 1172
[-0.41906757 -0.05260389  3.21053158 ...  0.         -0.22818595
 -0.11435678]
WU 1172
[ 2.12285913 -1.10570542 -0.44722942 ... -0.40840141 -0.55652609
  0.        ]
WY 1172
[ 2.12030562  1.88315316 -0.14218009 ... -0.60478649 -0.21109897
  0.72175335]
WYN 1172
[ 6.08192474 -1.31171559  2.13560338 ... -1.54556504  0.30071152
 -1.98688355]
WYNN 1172
[-0.18125431 -0.14526603 -1.58211124 ...  0.04950629  1.07458751
  1.53179197]
XEC 1172
[-1.18595825  0.19203553 -0.431241   ...  

In [149]:
returns

,AAL,AAPL,ABC,ABT,ADBE,ADI,ADM,ADS,ADSK,AEE,...,WYNN,XEC,XEL,XLNX,XOM,XRAY,XRX,YHOO,YUM,ZBH
0,11.320755,0.172887,-0.713474,-0.807929,1.644651,-0.157875,0.540197,0.197295,-1.519279,-0.396254,...,6.081925,-0.181254,-1.185958,-1.260835,0.390450,-1.188797,0.115875,0.760234,-0.341981,3.165608
1,-4.143126,-1.590633,-0.945537,0.555350,-0.212207,-0.189760,-0.252845,0.499850,0.237338,-0.687165,...,-1.311716,-0.145266,0.192036,-0.678372,0.864303,0.658834,-0.925926,-0.348230,-0.714893,-0.032293
2,2.946955,-0.184858,-1.603669,0.828423,-1.940457,-0.792142,-1.045624,0.919369,0.552486,-0.946825,...,2.135603,-1.582111,-0.431241,-1.004419,-0.314187,1.309047,0.467290,-2.737327,-0.028802,2.294019
3,-1.908397,0.664833,1.086531,0.511229,-0.542152,0.574899,-1.248799,1.075270,3.061224,-0.698533,...,-0.716475,1.866227,0.048123,1.461043,-0.401155,0.000000,-0.348837,0.000000,0.028810,-2.100439
4,-1.945525,-0.882160,1.113248,0.508629,-1.308258,-0.571613,0.064854,0.103428,-0.038081,0.814513,...,-0.324009,-1.287870,0.961996,-1.040000,1.121988,1.292143,2.450408,0.239515,1.728105,2.210032
5,0.793651,-1.137500,0.683371,-0.289176,-1.518915,-4.152025,-1.069345,-1.638378,-1.371432,-0.110169,...,0.443267,-1.451667,1.191043,-2.789010,-0.497875,-0.721026,0.797267,-0.358423,1.896945,-1.925508
6,7.874016,1.410553,1.960788,0.217513,1.738640,-0.033322,0.425807,-0.030006,1.390502,0.661757,...,2.088848,3.207155,1.553672,0.873185,-0.400285,1.396648,1.129944,1.318945,-0.277852,0.659800
7,2.007299,-0.579161,0.628691,0.108517,-1.047401,-0.100003,0.260926,0.105072,1.333333,0.365234,...,3.904898,1.517617,0.185447,-0.618298,0.014356,0.743804,-0.670391,1.301781,1.086648,0.287771
8,-1.610018,-1.671203,-1.065781,0.252932,-0.083574,-3.370037,-0.650615,-1.574453,-1.954887,-0.509461,...,-2.232701,-2.598325,-0.925502,-2.447121,-0.818024,-1.695376,-0.562430,-1.752342,-1.681367,-1.562251
9,2.545455,4.423831,1.671623,1.279511,3.010878,1.761050,0.000000,2.452774,0.345092,1.280179,...,1.163121,0.036543,0.887436,1.530608,0.231509,1.585533,2.036199,-0.416171,-0.056070,3.174087


In [152]:
returns.shape

(1172, 307)

In [161]:
cov_matrix = np.cov(returns,rowvar=False, bias=True) #covariance matrix for stocks

In [184]:
headers = ["symbol", "returns", "risk"]
ret_risk = pd.DataFrame(columns = headers)

In [171]:
aapl = {'symbol':"AAPL", 'returns':4, 'risk':5}
msft = {'symbol':"MSFT", 'returns':4, 'risk':0.66}

ret_risk.loc[len(ret_risk)] = aapl 
ret_risk.loc[len(ret_risk)] = msft 

ret_risk

,symbol,returns,risk
0,AAPL,4,5
1,AAPL,4,5
2,MSFT,4,0.66


In [185]:
def calculate_returns_risk():
    for s in returns:
        daily_returns=returns[s]
        exp_return = np.average(daily_returns)
        risk = np.std(daily_returns)
        new_dict =  {'symbol':s, 'returns':exp_return, 'risk':risk}
        ret_risk.loc[len(ret_risk)] = new_dict 

In [186]:
calculate_returns_risk()
ret_risk

,symbol,returns,risk
0,AAL,0.234597,3.347402
1,AAPL,0.117696,1.697128
2,ABC,0.098875,1.248235
3,ABT,0.046228,1.024431
4,ADBE,0.074223,1.878135
5,ADI,0.052577,1.531313
6,ADM,0.051086,1.534427
7,ADS,0.132472,1.656474
8,ADSK,0.088367,2.246610
9,AEE,0.037559,1.132059


In [227]:
ret_risk.to_csv(r'./nyse/Portfolio calculations/expected-return-risk.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [228]:
np.savetxt('./nyse/Portfolio calculations/covariance-matrix.csv', cov_matrix, delimiter=",")